# (Zhao et al., 2019) Safely and Quickly Deploying New Features with a Staged Rollout Framework Using Sequential Test and Adaptive Experimental Design

[Link to Paper](https://arxiv.org/pdf/1905.10493.pdf)

> It may be worth to first read the following articles first:
- https://engineering.atspotify.com/2023/03/choosing-sequential-testing-framework-comparisons-and-discussions/
- https://docs.statsig.com/experiments-plus/sequential-testing
- https://www.statsig.com/blog/sequential-testing-on-statsig

> Note that I focus on the sequential test more than the other parts of this paper.

# Abstract

Ths paper proposes a methodology for rolling out features in an automated way using an adaptive experimental design.

Powered by:
1. a continuous monitoring algorithm using a variant of the sequential probability ratio test (SPRT)
2. an automated ramp-up algorithm

3 Benefits:
1. for defective features, it can detect the regression early and reduce negative effect
2. for healthy features, it rolls out the feature quickly
3. it reduces the need for manual intervention via the automation of the feature rollout process.

# 1. Introduction

Describes the 5 properties for a safe feature rollout process.

The contribution of this paper includes:
- proposing a staged rollout framework for releasing new features autonomously by using a continuous monitoring algorithm and a ramp-up algorithm.
- introducing a scalable nonparametric variance estimator to be used with the sequential test to adjust the sample correlation when the observation is not independent.
- providing formulas for estimating power and required sample size for the sequential test.
- proposing and compared three ramp-up algorithms: time-based, power-based, and risk-based.
- evaluating the empirical performance of the staged rollout framework through examples.

# 2. Background

A variant of the sequential test is introduced, called a mixture sequential probability ratio test (mSPRT), used as the core monitoring algorithm. In addition, a nonparametric variance estimation method (delete-a-group jackknife) is introduced to correct the correlation for the sequential algorithm. The statistical power and sample size estimation is derived for the sequential test as well.

# 3. Staged Rollout Framework Overview

### A. Mixture Sequential Probability Ratio Test (mSPRT)

Sequential probability ratio test is widely used in clinicial research, where scientists often allow sample size dependent decisions to be made based on the likelihood ratio of two hypothesis.

The mSPRT introduced in (Johari et al., 2017) applies to A/B testing to enable multiple tests without inflating the false positive rate (FPR).

In an A/B test setting, assume the control variables $\{X_{\text{ctrl}}^{(i)}\}_{i=1}^{n_{\text{ctrl}}}$ are independent random variables from a distribution with a density function $f(x|\mu_{ctrl},\sigma_{ctrl})$ where the $\mu,\sigma$ represent mean and standard deviation.

Similarly, the distribution density function for the treatment variables $\{X_{\text{trt}}^{(i)}\}_{i=1}^{n_{\text{trt}}}$ is $f(x|\mu_{trt},\sigma_{trt})$.

The hypothesis to be tested is on the difference in distribution mean:

$$H_0:\delta:=\mu_{trt}-\mu_{ctrl}=\delta_0$$
$$H_1:\delta:=\mu_{trt}-\mu_{ctrl}\neq\delta_0$$

where $\delta$ represents the difference in mean between treatment and control, and $\delta_0$ is the difference value under null hypothesis ($\delta_0=0$ for testing if two groups have the same metric mean).

The test statistic used in mSPRT is the likelihood ratio integrated over a prior distribution of $\delta$ values under the alternative hypothesis. Denote the prior density function as $h(\delta)$, and for simplicity, a normal prior is chosen $h(\delta)\sim N(\delta_0,\tau)$ in this paper. It can be proven that the integrated likelihood ratio statistic is a Martingale under alternative hypothesis. The $(1-\alpha)$ confidence interval for $\delta$ is derived by (Pekelis, Walsh and Johari, 2015) as:

$$\bar x_{trt}-\bar x_{ctrl}\pm\sqrt{\frac{V(V+\tau)}{\tau}(-2\log (\alpha)-\log\left(\frac{V}{V+\tau}\right))}$$

with $\bar x_{trt}$ and $\bar x_{ctrl}$ as sample means and $V$ as the variance of sample mean difference (e.g., $V=\frac{v_{ctr}}{n_{ctrl}}+\frac{v_{trt}}{n_{trt}}$ where $v_{ctrl}$ and $v_{trt}$ are the sample variance estimates for control and treatment.)

### B. Variance Estimation

One assumption made by the sequential testing is the independence of each observation. However, this assumption often does not hold in practice. For example, if the click through rate is the metric of interest, it is improper to assume each impression is independent, since the same user can use the product multiple times on different days, and multiple impressions and clicks can be generated by the same user. Such observations are correlated since they are generated by the same user. Violating this independence assumption the sequential test can produce an inflated false positive rate.

Embedding a variance estimation with correlation correction is one way to generalize the mSPRT to correlated data. Several previous papers discuss the adjustment of metric variance in A/B testing. The delta method (Deng and Shi, 2016) and Bootstrap method (Bakshy and Eckles, 2013) are two variance estimation approaches that can be applied to correct the variance without the assumption of independence. The above require storing raw data, but in certain scenarios, a fast and scalable variance method is preferred. In this paper, it implements a version of the delete-a-group jacknife (Kott, 2001) variance estimation method.

To implement delete-a-group jacknife, the users (user ID) are split into $R$ partitions with equal probability using a hash function within each experiment group. The has function takes the user id as input and outputs an integer as the partition $\text{ID}\in\{1,2,...,R\}$.

The delete-a-group jacknife variance estimator can be expressed as

$$Var(\bar X)=\frac{R-1}{R}\sum_{r=1}^{R}(\bar X_{(r)}-\bar X)^2$$

with

$$\bar X_{(r)}=\frac{1}{\sum_{i=1}^nI(\text{hash}(\text{uuid}_i)\neq r)}\sum_{\{i:\text{hash}(\text{uuid}_i)\neq r\}}X_i$$

in other words, $\bar X_{(r)}$ is the metric mean for all users except partition $r$, $I(\cdot)$ is an indicator function with value $1$ if the argument condition is true and value $0$ otherwise. $\text{hash}(\cdot)$ is a hash function.

The delete-a-group jacknife variance estimation is scalable isnce only the partition level metrics are needed and stored for calculation, instead of the raw event-level data.

### C. Power and Sample Size Estimation

The statistical power of the sequential test is needed for evaluating when it is ready to fully ramp up and conclude the rollout.

To derive the power and sample size estimation formula, define the stopping time $N_\delta$ as the smallest sample size that the sequential test becomes significant given the true difference as $\delta$. Note $N_\delta$ is a random variable. The power of the sequential test for a finite sample size can be expressed as the probability of the sequential process becoming significant before the given sample size:

$$\Pr(N_\delta < N_{\delta_1-\beta}|\delta)=1-\beta$$

where $1-\beta$ indicates the power and $N_{\delta_1-\beta}$ denotes the sample size threshold for achieving the power $1-\beta$ given $\delta$.

Following the intuition that $N_{\delta_1-\beta}$ is the $1-\beta$ percentile of the distribution of $N_\delta$, it can be approximated by a linear combination of the distribution mean and standard deviation:

$$N_{\delta_1-\beta}\approx E[N_\delta]+f(\beta) * \sqrt{(\text{Var}(N_\delta))}$$
